In [17]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [18]:
# ------------------------------------------------------
# 📂 0. 데이터 로드 & 초기 정리
# ------------------------------------------------------
data_path = 'data/datasets.csv'
data = pd.read_csv(data_path)

In [19]:
# 불필요한 컬럼 제거
drop_cols = ['EmployeeNumber', 'EmployeeCount', 'Over18', 'StandardHours']
data.drop(columns=drop_cols, inplace=True)

In [20]:
# 타겟값 변환
data['Attrition'] = data['Attrition'].map({'Yes': 1, 'No': 0})

In [21]:
# ------------------------------------------------------
# ⚠️ 1. 이상값 및 결측 처리
# ------------------------------------------------------
# 0으로 나누기 방지용 NaN 처리
data['TotalWorkingYears'] = data['TotalWorkingYears'].replace(0, np.nan)
data['YearsAtCompany'] = data['YearsAtCompany'].replace(0, np.nan)

In [22]:
# ------------------------------------------------------
# 🎯 2. 커리어 관련 파생 변수
# ------------------------------------------------------
data['CareerStartAge'] = data['Age'] - data['TotalWorkingYears']                  # 첫 커리어 시작 나이
data['LoyaltyRatio'] = data['YearsAtCompany'] / data['TotalWorkingYears']        # 충성도 비율
data['PromotionDelay'] = data['YearsAtCompany'] - data['YearsSinceLastPromotion']# 승진까지 소요 기간


In [23]:
# ------------------------------------------------------
# 🏢 3. 근속 관련 파생 변수
# ------------------------------------------------------
data['CompanyLoyalty'] = data['YearsAtCompany'] / data['TotalWorkingYears']      # 회사 전체 근속 비율
data['RoleStability'] = data['YearsInCurrentRole'] / data['YearsAtCompany']      # 현재 직무 지속 비율
data['ManagerBond'] = data['YearsWithCurrManager'] / data['YearsAtCompany']      # 매니저 유대 비율

In [24]:
# ------------------------------------------------------
# 🔁 4. 이직/승진 경향
# ------------------------------------------------------
data['JobHopFrequency'] = data['NumCompaniesWorked'] / data['TotalWorkingYears'] # 연평균 이직 횟수
data['RecentlyPromoted'] = (data['YearsSinceLastPromotion'] <= 3).astype(int)     # 최근 3년 내 승진 여부

In [25]:
# ------------------------------------------------------
# 💰 5. 급여 및 성과 관련 변수
# ------------------------------------------------------
data['IncomePerYear'] = data['MonthlyIncome'] / data['TotalWorkingYears']        # 경력 대비 소득
data['HighPerformerQuit'] = ((data['PerformanceRating'] == 4) & 
                             (data['Attrition'] == 1)).astype(int)                # 고성과 이탈자 여부

In [26]:
# ------------------------------------------------------
# 🛣️ 6. 스트레스 및 이동 관련 변수
# ------------------------------------------------------
data['LongCommute'] = (data['DistanceFromHome'] > 20).astype(int)                # 장거리 출퇴근 여부
data['OverworkedPoorBalance'] = ((data['OverTime'] == 'Yes') & 
                                 (data['WorkLifeBalance'] <= 2)).astype(int)      # 워라밸 붕괴 여부

In [27]:
# ------------------------------------------------------
# 😊 7. 종합 만족도
# ------------------------------------------------------
data['OverallSatisfaction'] = (
    data['EnvironmentSatisfaction'] + 
    data['JobSatisfaction'] + 
    data['RelationshipSatisfaction']
) / 3                                                                             # 평균 만족도

In [28]:
# ------------------------------------------------------
# 🧑‍💼 8. 직급 기반 고위직 구분
# ------------------------------------------------------
manager_roles = ['Manager', 'Research Director']
data['Is_Manager'] = ((data['JobRole'].isin(manager_roles)) & 
                      (data['JobLevel'] >= 3)).astype(int)                        # 고위직 여부

In [29]:
# ------------------------------------------------------
# 🧹 9. 결측값 정리
# ------------------------------------------------------
data.fillna(0, inplace=True)

In [30]:
# ------------------------------------------------------
# ⚙️ 10. 컬럼 타입 분리
# ------------------------------------------------------
numeric_features = data.select_dtypes(include=['int64', 'float64']).drop(columns=['Attrition']).columns.tolist()
categorical_features = data.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# ------------------------------------------------------
# 🧪 11. 전처리 파이프라인 구성 및 적용
# ------------------------------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)